In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [125]:
order_df = pd.read_csv("order_items.csv")

In [126]:
df_train = pd.read_csv("train_trips.csv")

In [127]:
order_df.columns

Index(['trip_id', 'item_id', 'department_name', 'quantity'], dtype='object')

## Extracting Features from Order details ...
1. total_quantity per trip id, 
2. departmentwise quantity per trip id, 
3. num distict items per trip id,
4. number of distinct departments per trip id

In [128]:
##1. One trip id is having multiple order from multiple deparments . we can have total quantity as one of the feature

total_quantity_trip = order_df.groupby(by=['trip_id'])["quantity"].sum()

# to Convert pd.series into data frame and then inner join to get total quantity for a trip
total_quantity_trip = total_quantity_trip.to_frame().reset_index()

total_quantity_trip.rename(columns={"quantity": "total_quantity"}, inplace=True)


total_quantity_trip.head()

,trip_id,total_quantity
0,3119513,183.0
1,3119516,9.0
2,3119518,21.0
3,3119519,39.0
4,3119520,36.0


In [129]:
##2. Departmentwise quantity per trip id, 
order_quantity_by_department = order_df.groupby(by=['trip_id','department_name'])["quantity"].sum()
order_quantity_by_department = order_quantity_by_department.to_frame().reset_index()


orders_pivot = order_quantity_by_department.pivot_table(index='trip_id',columns='department_name',values='quantity')

orders_pivot.to_csv('orders_pivot.csv') 


In [130]:
orders = pd.read_csv('orders_pivot.csv')
orders = orders.fillna(0)
df_train = pd.merge(df_train, orders,  on = "trip_id",how='inner')

df_train.head(5)

,trip_id,shopper_id,fulfillment_model,store_id,shopping_started_at,shopping_ended_at,25% OFF Supplements 8/14-8/16,4th of July,Academy Awards,Alcohol,...,Summer Drinks,Super Bowl Party,TEST SPECIAL AISLE,Thanksgiving,Thanksgiving A to Z,Thanksgiving Menu,Travel,Valentine's Day Specials,Valentine's- Made with Love!,Vitamins & Supplements
0,3119519,48539,model_1,6,2015-09-01 07:03:56,2015-09-01 07:30:56,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3119513,3775,model_1,1,2015-09-01 07:04:33,2015-09-01 07:40:33,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3119516,4362,model_1,1,2015-09-01 07:23:21,2015-09-01 07:41:21,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3119792,47659,model_1,1,2015-09-01 07:29:52,2015-09-01 08:55:52,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3119922,11475,model_1,1,2015-09-01 07:32:21,2015-09-01 09:01:21,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
##3. Distinct items per trip id 

num_distinct_items_trip = order_df.groupby(by=['trip_id'])["item_id"].nunique()

# to Convert pd.series into data frame and then inner join to get total quantity for a trip
num_distinct_items_trip = num_distinct_items_trip.to_frame().reset_index()

num_distinct_items_trip.rename(columns={"item_id": "distinct_items"}, inplace=True)

num_distinct_items_trip.head()

,trip_id,distinct_items
0,3119513,33
1,3119516,5
2,3119518,12
3,3119519,26
4,3119520,33


In [134]:
##4. Department wise quantiy per trip id 

num_distinct_department_trip = order_df.groupby(by=['trip_id'])["department_name"].nunique()

# to Convert pd.series into data frame and then inner join to get total quantity for a trip
num_distinct_department_trip = num_distinct_department_trip.to_frame().reset_index()

num_distinct_department_trip.rename(columns={"department_name": "distinct_departments"}, inplace=True)
num_distinct_department_trip.head()

,trip_id,distinct_departments
0,3119513,8
1,3119516,5
2,3119518,6
3,3119519,11
4,3119520,14


In [135]:
df_train = pd.merge(df_train, total_quantity_trip,  on = "trip_id",how='inner')
df_train = pd.merge(df_train, num_distinct_items_trip,  on = "trip_id",how='inner')
df_train = pd.merge(df_train, num_distinct_department_trip,  on = "trip_id",how='inner')

In [136]:
df_train.columns

Index(['trip_id', 'shopper_id', 'fulfillment_model', 'store_id',
       'shopping_started_at', 'shopping_ended_at',
       '25% OFF Supplements 8/14-8/16', '4th of July', 'Academy Awards',
       'Alcohol', 'All It Takes to Bake!', 'Aquatics', 'BBQ Favorites',
       'Babies', 'Bakery', 'Beverages', 'Bi-Rite Creamery',
       'Bi-Rite's Winter Favorites!', 'Bird', 'Breakfast', 'Bulk',
       'Burning Man Essentials', 'Business Items', 'Buyers' Picks',
       'Canned Goods', 'Cat', 'Cheese', 'Christmas & NYE Menu', 'Costumes',
       'Dairy & Eggs', 'Deli', 'Dog', 'Dry Goods & Pasta',
       'Fall Wine, Beer, & Spirits', 'Father's Day', 'Fill the Grill',
       'Find it at Target', 'FitMarket', 'Floral', 'Flowers & Plants',
       'Fromagerie', 'Frozen', 'Game Day Specials', 'Gifts for the Foodie',
       'Halloween', 'Hanukkah', 'Hanukkah Menu', 'Healthy Choices for 2016',
       'Healthy Eating', 'Holiday', 'Holiday Butcher', 'Holiday Essentials',
       'Holiday Favorites', 'Holiday 